In [ ]:
from google.colab import drive
import sys
drive.mount('/content/gdrive')
sys.path.append('/content/gdrive/MyDrive/AIM')
!ls -l '/content/gdrive/MyDrive/AIM/*.jpg'
import aim_util

Mounted at /content/gdrive
total 48574
-rw------- 1 root root    1108 May  2 18:51 aim_util.py
-rw------- 1 root root    2696 Apr 15 14:07 citizen_1864_20240414_213308040_groundtruth.txt
-rw------- 1 root root 3663644 Apr 14 21:34 citizen_1864_20240414_213308040.jpg
-rw------- 1 root root    2696 May  2 20:40 citizen_1864_groundtruth_annotated.txt
-rw------- 1 root root    3899 May  3 15:25 citizen_1864_groundtruth_word_list_lower.txt
-rw------- 1 root root    3899 May  3 15:25 citizen_1864_groundtruth_word_list.txt
-rw------- 1 root root    3899 May  3 15:25 citizen_1864_groundtruth_word_string_lower.txt
-rw------- 1 root root    3899 May  3 15:25 citizen_1864_groundtruth_word_string.txt
-rw------- 1 root root 3663644 Apr 23 20:54 citizen_1864_rescan.jpg
-rw------- 1 root root   68979 May  3 18:44 citizen_1864_xform0_carlv_bbox.txt
-rw------- 1 root root    3778 May  3 18:44 citizen_1864_xform0_carlv_word_list.txt
-rw------- 1 root root    2425 May  3 18:44 citizen_1864_xform0_carlv_w

In [ ]:
!ls -l /content/gdrive/MyDrive/AIM/*_word_list.txt

-rw------- 1 root root 3899 May  3 15:25 /content/gdrive/MyDrive/AIM/citizen_1864_groundtruth_word_list.txt
-rw------- 1 root root 3778 May  3 18:44 /content/gdrive/MyDrive/AIM/citizen_1864_xform0_carlv_word_list.txt
-rw------- 1 root root 3796 May  3 18:48 /content/gdrive/MyDrive/AIM/citizen_1864_xform1_carlv_word_list.txt
-rw------- 1 root root 3044 May  3 18:51 /content/gdrive/MyDrive/AIM/citizen_1864_xform2_carlv_word_list.txt
-rw------- 1 root root 3770 May  3 18:55 /content/gdrive/MyDrive/AIM/citizen_1864_xform3_carlv_word_list.txt
-rw------- 1 root root 2450 May  3 19:22 /content/gdrive/MyDrive/AIM/citizen_1864_xform4_carlv_word_list.txt
-rw------- 1 root root 3809 May  3 19:25 /content/gdrive/MyDrive/AIM/citizen_1864_xform5_carlv_word_list.txt


In [ ]:
import ast

groundtruth_path = '/content/gdrive/MyDrive/AIM/citizen_1864_groundtruth_word_list_lower.txt'
pred_xform0_path = '/content/gdrive/MyDrive/AIM/citizen_1864_xform0_carlv_word_list.txt'
pred_xform1_path = '/content/gdrive/MyDrive/AIM/citizen_1864_xform1_carlv_word_list.txt'
pred_xform2_path = '/content/gdrive/MyDrive/AIM/citizen_1864_xform2_carlv_word_list.txt'
pred_xform3_path = '/content/gdrive/MyDrive/AIM/citizen_1864_xform3_carlv_word_list.txt'
pred_xform4_path = '/content/gdrive/MyDrive/AIM/citizen_1864_xform4_carlv_word_list.txt'
pred_xform5_path = '/content/gdrive/MyDrive/AIM/citizen_1864_xform5_carlv_word_list.txt'

def file_to_list(path):
    with open(path, 'r') as f:
        target_list = ast.literal_eval(f.read())
    return target_list

groundtruth_list = file_to_list(groundtruth_path)
pred_xform0_list = file_to_list(pred_xform0_path)
pred_xform1_list = file_to_list(pred_xform1_path)
pred_xform2_list = file_to_list(pred_xform2_path)
pred_xform3_list = file_to_list(pred_xform3_path)
pred_xform4_list = file_to_list(pred_xform4_path)
pred_xform5_list = file_to_list(pred_xform5_path)

In [ ]:
!pip install fuzzywuzzy

In [ ]:
# fixes multiple groundtruth word matching e.g. said, said, said, said
import fuzzywuzzy.fuzz

def rate_match(groundtruth_list, prediction_list):
  print(f"groundtruth_list => {groundtruth_list}")
  print(f"prediction list => {prediction_list}")

  # Partial match threshold (experiment with this value)
  PARTIAL_MATCH_THRESHOLD = 80

  # Find matches, partial matches, and mismatches
  match_list = []
  partial_match_list = []
  mismatch_list = []

  # Initialize counts and tracking dictionaries
  matches = 0
  partial_matches = 0
  groundtruth_counts = {word: groundtruth_list.count(word) for word in groundtruth_list}
  predicted_counts = {word: prediction_list.count(word) for word in prediction_list}

  for word in prediction_list:
      best_match_found = False
      partial_match_found = False

      for groundtruth_word in groundtruth_list:
          ratio = fuzzywuzzy.fuzz.ratio(word, groundtruth_word)

          if ratio == 100 and groundtruth_counts[groundtruth_word] > 0:
              matches += 1
              match_list.append(word)
              groundtruth_counts[groundtruth_word] -= 1  # Decrement count for matched word
              best_match_found = True
              break

      if not best_match_found:
          for groundtruth_word in groundtruth_list:
              ratio = fuzzywuzzy.fuzz.ratio(word, groundtruth_word)

              if ratio >= PARTIAL_MATCH_THRESHOLD and groundtruth_counts[groundtruth_word] > 0:
                  partial_matches += 1
                  partial_match_list.append(word)
                  groundtruth_counts[groundtruth_word] -= 1
                  partial_match_found = True
                  break
      if not best_match_found and not partial_match_found:
          mismatch_list.append(word)

  # Calculate percentages more accurately
  exact_match_percentage = (matches / len(groundtruth_list)) * 100 if groundtruth_list else 0
  partial_match_percentage = (partial_matches / len(groundtruth_list)) * 100 if groundtruth_list else 0

  # Print the results
  print("Groundtruth list length:", len(groundtruth_list))
  print("Prediction list length:", len(prediction_list))
  print(f"Exact match percentage: {exact_match_percentage}% with exact match count {matches}")
  print(f"Partial match percentage: {partial_match_percentage}% with exact match count {partial_matches}")
  #print("Partial match percentage:", partial_match_percentage, "%")
  print("\nMatching words:")
  print(match_list)
  print("\nPartially Matching words:")
  print(partial_match_list)
  print("\nMismatched words:")
  print(mismatch_list)

  return matches, partial_matches, exact_match_percentage, partial_match_percentage

/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
print(f"==========xform0===========")
rate_match(groundtruth_list, pred_xform0_list)
print(f"==========xform1===========")
rate_match(groundtruth_list, pred_xform1_list)
print(f"==========xform2===========")
rate_match(groundtruth_list, pred_xform2_list)
print(f"==========xform3===========")
rate_match(groundtruth_list, pred_xform3_list)
print(f"==========xform4===========")
rate_match(groundtruth_list, pred_xform4_list)
print(f"==========xform5===========")
rate_match(groundtruth_list, pred_xform5_list)

==========xform0===========
groundtruth_list => ['united', 'states', 'of', 'america', 'our', 'liberty', 'we', 'prize', 'and', 'out', 'rights', 'we', 'will', 'maintain', 'state', 'of', 'iowa', 'ss:', 'scott', 'county', 'be', 'it', 'remembered', 'that', 'at', 'a', 'term', 'of', 'the', 'district', 'court', 'holden', 'in', 'and', 'for', 'said', 'county', 'in', 'the', 'city', 'of', 'davenport', 'therein', 'on', 'the', '7', 'day', 'of', 'november', 'in', 'the', 'year', 'of', 'our', 'lord', 'one', 'thousand', 'eight', 'hundred', 'sixtyfour', 'was', 'presentthe', 'honorable', 'j.', 'scott', 'richman', 'sole', 'presiding', 'judge', 'john', 'm.', 'lyter', 'sheriff', 'of', 'said', 'county', 'and', 'ira', 'm.', 'gifford', 'clerk', 'of', 'said', 'court', 'when', 'the', 'fol-', 'lowing', 'among', 'other', 'proceedings', 'were', 'had', 'to', 'wit:', 'hans', 'claus', 'olson', 'a', 'native', 'of', 'norway', 'and', 'at', 'present', 'residing', 'within', 'said', 'state', 'appeared', 'in', 'open', 'court'

(312, 49, 68.72246696035242, 10.79295154185022)

In [ ]:
!pip install random_string_detector

In [ ]:
from random_string_detector import RandomStringDetector

detector = RandomStringDetector()

def scrub_pred_list(pred_list):
  keyword_list = []
  gibberish_list = []

  for s in pred_list:
      print(s)
      if detector(s) or len(s)<2:
          gibberish_list.append(s)
      else:
          keyword_list.append(s)

  print("Keyword list:", keyword_list)
  print("Gibberish list:", gibberish_list)

  return keyword_list

In [ ]:
# pred_list = ['united', 'states', 'of', 'america', 'luere', 'ralling', 'o', 'o', 'v', 'state', 'of', 'iowa', 'sss', 'co', 'scott', 'cunty', 'be', 'it', 'renteutbeted', 'that', 'at', 'e', 'term', 'or', 'the', 'district', 'court', 'holden', 'n', 'and', 'for', 'said', 'county', 'n', 'the', 'e', 'ovcm', 'elc', 'city', 'of', 'davenport', 'theein', 'on', 'the', 'day', 'of', 'in', 'the', 'year', 'of', 'our', 'lord', 'one', 'o', 'susty', 'thousand', 'eight', 'hundre', 'and', 'fuw', 'was', 'present', 'the', 'hororable', 'je', 'scott', 'richmany', 'sole', 'prsiding', 'judge', 'john', 'me', 'lyter', 'sheriff', 'of', 'said', 'county', 'and', 'ira', 't', 'gifford', 'clerk', 'of', 'said', 'court', 'whenthe', 'fol', 'v', 'anws', 'cans', 'olen', 'lowing', 'among', 'other', 'roceedings', 'were', 'had', 'towits', 's', 'a', 'native', 'of', 'a', 'loc', 'prcsens', 'resian', 'vltmm', 'said', 'stater', 'appeared', 'lile', 'open', 'cout', 'icctolente', 'be', 'aemitted', 'oeceme', 'v', 'a', 'citizen', 'of', 'thie', 'uited', 'states', 'oww', 'and', 'it', 'roadwo', 'appearing', 'to', 'cot', 'the', 'oo', 'satifaction', 'of', 'ssked', 'the', 'court', 'that', 'oouns', 'he', 'had', 'declaredn', 'oath', 'c', 'before', 'e', 'a', 'court', 'gf', 'record', 'heing', 'common', 'law', 'jurisdiction', 'and', 'using', 'cl', 'sal', 'two', 'years', 'at', 'least', 'before', 'his', 'admissions', 'that', 'it', 'e', 'was', 'bona', 'fide', 'his', 'inter', 'tion', 'to', 'become', 'a', 'citizen', 'of', 'the', 'united', 'states', 'and', 'to', 'renounce', 'forever', 'all', 'allegiance', 'to', 'cas', 'o', 'foreign', 'prince', 'potentater', 'state', 'or', 'sovereigntys', 'whatsoever', 'and', 'particularly', 'to', 'iw', 'wedens', 'o', 'subile', 'of', 'wh', 'he', 'was', 'heretofore', 'a', 'and', 'laid', 'applicant', 'having', 'declared', 'on', 'oath', 'before', 'this', 'co', 'court', 'that', 'he', 'will', 'support', 'the', 'constitution', 'of', 'the', 'united', 'statess', 'nd', 'that', 'he', 'doth', 'absolutely', 'and', 'entirely', 'renounce', 's', 'and', 'abjure', 'all', 'allegiance', 'and', 'fidelity', 'to', 'every', 'foreign', 'princer', 'poentater', 'state', 'or', 'sovereigntys', 'whatsoever', 'nd', 'pars', 'ticularly', 'to', 'the', 'power', 'above', 'named', 'the', 'court', 'being', 'satisfied', 'that', 'said', 'applicant', 'has', 'resided', 'within', 'the', 'united', 'o', 'states', 'for', 'the', 'term', 'offive', 'years', 'next', 'preceeding', 'his', 'admission', 'without', 'being', 'at', 'any', 'time', 'during', 'the', 'said', 'fre', 'years', 'out', 'of', 'the', 'territory', 'of', 'the', 'united', 'states', 'and', 'within', 'this', 'state', 'onc', 'year', 'at', 'leasty', 'and', 'it', 'further', 'appearing', 'to', 'the', 'n', 'n', 'satisfaction', 'of', 'this', 'court', 'that', 'during', 'that', 'time', 'he', 'has', 'behaved', 'as', 'a', 'man', 'of', 'good', 'moral', 'character', 'attached', 'to', 'the', 'n', 'constitution', 'of', 'the', 'united', 'states', 'and', 'well', 'disposed', 'to', 'the', 'good', 'order', 'and', 'happiness', 'of', 'the', 'same', 'thereuon', 'the', 'court', 'admitted', 'the', 'sald', 'applicant', 'to', 'become', 'a', 'citizen', 'of', 'tho', 'unitod', 'states', 'and', 'ordered', 'all', 'proceedings', 'aoresaid', 'eo', 'to', 'e', 'be', 'entere', 'dl', 'eeo', 'llic', 'cout', 's', 'y', 'testimotty', 'aleeon', 'l', 'ira', 'm', 'giffordy', 'olerk', 'of', 'the', 'court', 'aforesaul', 'have', 'si', 'hereto', 'set', 'my', 'hand', 'and', 'aflxed', 'the', 'seal', 'of', 'sauid', 'court', 'at', 'ofice', 'in', 'hes', 'city', 'e', 'of', 'dawenport', 'in', 'said', 'county', 'this', 'the', 'seen', 'day', 's', 'nnem', 'bes', 'in', 'the', 'year', 'of', 'ourlord', 'one', 'thousanl', 'eight', 'hundred', 'and', 'sixty', 'sangezc', 'clerk', 'f', 'district', 'court', 'n']

# print(f"==========unfiltered===========")
# rate_match(groundtruth_list, pred_list)

# print(f"==========no nonsense===========")
# rate_match(groundtruth_list, keyword_list)

In [ ]:
print(pred_xform2_list)
pred_list = scrub_pred_list(pred_xform2_list)
print(pred_list)

['united', 'staes', 'of', 'america', 'state', 'iowa', 'ss', 'scott', 'cfuntys', 't', 'ineueutberess', 'that', 'at', 'c', 'term', 'of', 'the', 'district', 'court', 'holden', 'in', 'and', 'for', 'said', 'county', 'rathe', 'city', 'of', 'datenport', 'the', 'eln', '011', 'the', 'f', 'day', 'of', 'ovcme', 'ess', 'in', 'the', 'year', 'of', 'our', 'led', 'fre', 'musl', 'thousand', 'eight', 'hundrdand', 'fums', 'was', 'present', 'the', 'bloorable', 'jl', 'scott', 'richman', 'sole', 'priding', 'tudge', 'john', 'me', 's', 'ter', 'sheriff', 'of', 'said', 'county', 'and', 'ira', 'm', 'gifford', 'clerk', 'of', 'said', 'court', 'wheihe', 'fol', 'aren', 'lowing', 'among', 'hadt', 'towit', 'lt', 'appear', 'catte', 'elarede', 'cos', 'aea', 'ele', 'dli', 'tecord', 'law', 'jurisdiction', 'andusing', 'cl', 'sal', 'anears', 'at', 'was', 'bona', 'fide', 'his', 'intertion', 'to', 'become', 'el', 'citizen', 'of', 'the', 'united', 'stat', 'and', 'eoarenounce', 'forever', 'sali', 'allegiance', 'foreign', 'princ

ZeroDivisionError: division by zero

In [ ]:
print(f"==========xform0===========")
pred_list = scrub_pred_list(pred_xform0_list)
rate_match(groundtruth_list, pred_list)
print(f"==========xform1===========")
pred_list = scrub_pred_list(pred_xform1_list)
rate_match(groundtruth_list, pred_list)
# print(f"==========xform2===========")
# pred_list = scrub_pred_list(pred_xform2_list)
# rate_match(groundtruth_list, pred_list)
print(f"==========xform3===========")
pred_list = scrub_pred_list(pred_xform3_list)
rate_match(groundtruth_list, pred_list)
# print(f"==========xform4===========")
# pred_list = scrub_pred_list(pred_xform4_list)
# rate_match(groundtruth_list, pred_list)
print(f"==========xform5===========")
pred_list = scrub_pred_list(pred_xform5_list)
rate_match(groundtruth_list, pred_list)

==========xform0===========
Keyword list: ['united', 'states', 'of', 'america', 'luere', 'ralling', 'state', 'of', 'iowa', 'sss', 'co', 'scott', 'cunty', 'be', 'it', 'renteutbeted', 'that', 'at', 'term', 'or', 'the', 'district', 'court', 'holden', 'and', 'for', 'said', 'county', 'the', 'elc', 'city', 'of', 'davenport', 'theein', 'on', 'the', 'day', 'of', 'in', 'the', 'year', 'of', 'our', 'lord', 'one', 'susty', 'thousand', 'eight', 'hundre', 'and', 'fuw', 'was', 'present', 'the', 'hororable', 'je', 'scott', 'richmany', 'sole', 'prsiding', 'judge', 'john', 'me', 'lyter', 'sheriff', 'of', 'said', 'county', 'and', 'ira', 'gifford', 'clerk', 'of', 'said', 'court', 'whenthe', 'fol', 'anws', 'cans', 'olen', 'lowing', 'among', 'other', 'roceedings', 'were', 'had', 'towits', 'native', 'of', 'loc', 'prcsens', 'resian', 'vltmm', 'said', 'stater', 'appeared', 'lile', 'open', 'cout', 'icctolente', 'be', 'aemitted', 'oeceme', 'citizen', 'of', 'thie', 'uited', 'states', 'oww', 'and', 'it', 'roadwo',

(302, 49, 72.77108433734941, 11.80722891566265)